In [6]:
import pymysql
import colorExtract

In [ ]:
def insertColor(color, id):
    conn = pymysql.connect(host='127.0.0.1', user='ssafy', password='ssafy', db='kkalong', charset='utf8')
    cur = conn.cursor()
    sql = "update clothing set color = %s where clothing_id = %s"
    cur.execute(sql, (color, id));
    conn.commit()
    conn.close()

In [ ]:
conn = pymysql.connect(host='127.0.0.1', user='ssafy', password='ssafy', db='kkalong', charset='utf8')
cur = conn.cursor()
cur.execute("select * from clothing");

colorArr = []
for row in cur:
    img = row[5]
    id = row[0]
    print(row[0])
    color = colorExtract.image_preprocess(img)
    colorArr.append([color, id])
    print(color)

conn.commit()
conn.close()

In [31]:
for row in colorArr:
    insertColor(row[0], row[1])

[['민트', '스카이 블루', '네온 블루', '라벤더', '연청'], ['네이비', '보라색', '데님', '파란색', '중청', '진청'], ['화이트', '아이보리', '샌드', '베이지', '페일 핑크', '라이트 그레이', '회색'], ['네온 그린', '라이트 그린', '녹색', '올리브 그린', '카키', '다크 그린', '카키 베이지', '라이트 옐로우', '노란색', '머스타드', '카멜', '라이트 오렌지', '네온 오렌지', '주황색', '레드 브라운'], ['딥레드', '빨간색', '자주', '버건디', '갈색', '로즈 골드', '라즈베리', '네온 핑크', '분홍색', '라이트 핑크', '피치', '오렌지 핑크'], ['화이트', '아이보리', '샌드', '베이지', '페일 핑크', '라이트 그레이', '회색']]


In [3]:
import pymysql
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

In [4]:
conn = pymysql.connect(host='127.0.0.1', user='ssafy', password='ssafy', db='kkalong', charset='utf8')
sql = "select * from clothing"
df = pd.read_sql_query(sql, conn)
# data = pd.read_sql_table('clothing', conn)
print(df[df[df['style']=='casual']['main_category'] == 1]['sub_category'])

5     101
11    104
13    104
15    102
21    104
22    108
27    106
28    104
34    102
39    104
42    107
46    105
49    104
54    105
Name: sub_category, dtype: int64


C:\Users\jws48\AppData\Local\Temp\ipykernel_28376\57397361.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


In [1]:
import recommendCodi
recommendCodi.weatherRecommend('casual', 'F', 'fall', 29)

{'top': 1, 'bottom': 2, 'outer': 0, 'shoes': 6, 'bag': 7, 'hat': 0}
{'top': ['라이트 그레이', '회색', '다크 그레이'], 'bottom': ['네이비', '보라색', '데님', '파란색', '중청', '진청'], 'outer': ['블랙', '흑청'], 'shoes': ['네이비', '보라색', '데님', '파란색', '중청', '진청'], 'bag': [], 'hat': []}
F
casual
fall
29


'a'

In [1]:
import recommendCodi
print(recommendCodi.top_summer)

[101, 102, 103, 104]


In [13]:
import random
a = random.choices(range(1,3), weights=[0.3, 0.3])[0]
a

2

In [1]:
import recommendCodi
recommendCodi.personalRecommend('casual', 'F', 'fall', 'summer')

{'top': 1, 'bottom': 2, 'outer': 5, 'shoes': 6, 'bag': 0, 'hat': 0}
{1: 108, 2: 203, 3: 0, 4: 0, 5: 512, 6: 609, 7: 0, 8: 0}
{'top': 1, 'bottom': 2, 'outer': 5, 'shoes': 6, 'bag': 0, 'hat': 0}
진청


'b'